## Zadanie domowe: Projektowanie filtru metodą okien

Zauważone w zadaniu z filtracją artefakty są efektem zastosowania filtru idealnego (wycięcia odpowiednich częstotliwości).
Aby je usunąć, należy filtr idealny w dziedzinie przestrzennej przemnożyć przez dwuwymiarowe okno (stworzone na podstawie jednowymiarowego).

1. Zdefiniuj rozmiar filtra (np. 21).

2. Stwórz okno np. Hamminga (`np.Hmming`) o zadanym rozmiarze.

3. Na podstawie tego okna stwórz okno dwuwymiarowe: $F = f^Tf$.
Można to zrealizować prostą instrukcją:
```python
        hanning2d = np.outer(hanning, hanning)
```

4. Stwórz wzorcowy filtr idealny tak samo jak w poprzednim ćwiczeniu.
Jego rozmiar musi być taki sam jak zdefiniowany w punkcie pierwszym.

5. Generalna zasada jest taka, że należy obliczyć odwrotną transformatę Fouriera filtru idealnego.
Aby zagwarantować jednak dobrą lokalizację i symetryczność tworzonego filtru wykorzystuje się dodatkowe rotacje i przesunięcia.
```python
        FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
        FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
        FilterFidft = cv2.idft(np.float32(FilterFRot3),
                               flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
        FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)
```

6. Aby otrzymać poprawny filtr należy teraz pomnożyć dwuwymiarowe okno ze stworzonym filtrem idealnym w dziedzinie przestrzennej.

7. Stworzony filtr powinien zostać znowu przeniesiony do dziedziny częstotliwości.
Należy jednak rozszerzyć go tak, aby miał taki sam rozmiar jak filtrowany obraz.
Operację tę wykonaj tak samo jak w poprzednim ćwiczeniu (`cv2.copyMakeBorder`).
Oblicz transformatę Fouriera i wykonaj przesunięcie.
Następnie oblicz wartość bezwzględną (dzięki temu filtr nie będzie wpływał na fazę).
Wynikiem jest poprawiony filtr w dziedzinie częstotliwości.

8. Przeprowadź filtrację obrazu *lena.bmp* za pomocą zaprojektowanego filtra.
Wyświetl filtr, oraz wynik filtracji.
Do wyświetlenia filtru możesz użyć przestrzeń częstotliwości wygenerowaną w poprzednim zadaniu (*FSpaceRowsM* i *FSpaceColsM*).

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np
import math

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/'

fileNames = ["lena.bmp"]
for fileName in fileNames:
  path = 'img/' + fileName
  if not os.path.exists(path):
      r = requests.get(url + fileName, allow_redirects=True)
      open(path, 'wb').write(r.content)


In [ ]:
def show(imgs, titles=None):
    if len(imgs) == 1:
        plt.figure(figsize=(10, 10))
        plt.imshow(imgs[0], cmap='gray')
        if titles is not None:
            plt.title(titles[0])
        plt.axis('off')
        plt.show()
        return

    _, axis = plt.subplots(1, len(imgs), figsize=(5*len(imgs), 5))
    for i, ax in enumerate(axis):
        ax.imshow(imgs[i], cmap='gray')
        if titles is not None:
            ax.set_title(titles[i])
        ax.axis('off')

    plt.show()

In [ ]:
def window_filter(img, flag, threshold, size=21):
    size_ = int(size // 2)
    hamming = np.hamming(size)
    hamming2d = np.outer(hamming, hamming)
    
    FSpaceRows = 2 * np.fft.fftshift(np.fft.fftfreq(hamming2d.shape[0]))
    FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, hamming2d.shape[1]]))
    FSpaceCols = 2 * np.fft.fftshift(np.fft.fftfreq(hamming2d.shape[1]))
    FSpaceColsM = np.outer(np.ones([1, hamming2d.shape[0]]), FSpaceCols)
    FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))
    
    # Filter selection
    match flag:
        case 0 | 'dolnoprzepustowy':
            FilterF = FreqR <= threshold
        case 1 | 'górnoprzepustowy':
            FilterF = FreqR > threshold
        case 2 | 'pasmowoprzepustowy':
            low_threshold = threshold[0]
            high_threshold = threshold[1]
            FilterF = (FreqR >= low_threshold) & (FreqR <= high_threshold)
        case _:
            FilterF = FreqR
    
    FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
    FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
    FilterFidft = cv2.idft(np.float32(FilterFRot3),
                           flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
    FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)
    
    filter_result = hamming2d * FilterFI
    
    up = 0
    down = img.shape[0] - filter_result.shape[0]
    left = 0
    right = img.shape[1] - filter_result.shape[1]
    filter_result_padded = cv2.copyMakeBorder(filter_result, up, down, left, right, cv2.BORDER_CONSTANT, value=0)
    filter_result_fft = np.fft.fft2(filter_result_padded)
    filter_result_fft_shifted = np.fft.fftshift(filter_result_fft)
    filter_result_magnitude = np.abs(filter_result_fft_shifted)
    
    img_fft = np.fft.fft2(img)
    img_fft_shifted = np.fft.fftshift(img_fft)
    
    img_filtered_fft = img_fft_shifted * filter_result_magnitude
    img_filtered = np.fft.ifft2(np.fft.ifftshift(img_filtered_fft))
    img_filtered = np.abs(img_filtered)
    
    return img_filtered

In [ ]:
img = cv2.imread('img/lena.bmp', cv2.IMREAD_GRAYSCALE)
img_filtered = window_filter(img, 'dolnoprzepustowy', 30, 21)

show([img, img_filtered, abs(img - img_filtered)], ['Original', 'Filtered', 'Difference'])
show([img])
show([img_filtered])